In [1]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name=ws.name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-142444
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-142444


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"
compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [35]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': uniform(1,50)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.25,evaluation_interval=2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".",
                      entry_script="train.py",
                      compute_target="cpu-cluster")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling = ps,
    policy=policy,
    primary_metric_name='accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run = exp.submit(config=hyperdrive_config)
RunDetails(run).show()
run.wait_for_completion(show_output=True)


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c9baaef9-3ce2-40bc-bac9-71c744bd4043
Web View: https://ml.azure.com/runs/HD_c9baaef9-3ce2-40bc-bac9-71c744bd4043?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-142444/workspaces/quick-starts-ws-142444&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-12T00:42:52.804401][API][INFO]Experiment created<END>\n""<START>[2021-04-12T00:42:53.700292][GENERATOR][INFO]Trying to sample '20' jobs from the hyperparameter space<END>\n""<START>[2021-04-12T00:42:54.017892][GENERATOR][INFO]Successfully sampled '20' jobs, they will soon be submitted to the execution target.<END>\n"


In [29]:
import joblib

# Get your best run and save the model from that run.
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('ID: ',best_mod.id,' Accuary:', best_mod_metrics['accuracy'])
best_run.get_file_names()
#best_run.download_file(
 #   best_run.get_file_names()[-1],
  #  output_file_path="./output/"
#)


ID:  HD_a1d4cb02-0661-411e-ac08-bae17fe38f6a_2  Accuary: 0.9121396054628225


['azureml-logs/55_azureml-execution-tvmps_4c66567ef7cfed41e1f2a522d4335f4a9f971f08cacf2039f281d087b7e66b68_d.txt',
 'azureml-logs/65_job_prep-tvmps_4c66567ef7cfed41e1f2a522d4335f4a9f971f08cacf2039f281d087b7e66b68_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_4c66567ef7cfed41e1f2a522d4335f4a9f971f08cacf2039f281d087b7e66b68_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/104_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log']

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(
    path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv",
    separator=",")


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric='accuracy',
    training_data=ds,
    label_column_name=y,
    n_cross_validations=5)

In [ ]:
# Submit your automl run

automl_exp = experiment(ws,"AutoML-exp")
automl_run = automl_exp.submit(config=automl_config,show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###